In [1]:
import numpy as np
import pandas as pd

# install pandas_ta - https://github.com/twopirllc/pandas-ta
import pandas_ta as ta

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, SGDClassifier

# Algorithmic Trading Bot

## 0. Step-by-step guide to building an algorithmic trading bot for Bitcoin

Here's a step-by-step guide to building an algorithmic trading bot for Bitcoin, utilizing machine learning to identify buy and sell signals.

### Step 0.1: Data Collection

First, gather historical Bitcoin price data, which will be used for model training. Ideally, this data should include:

1. **Price Data**: Open, close, high, and low prices over specific time intervals (e.g., daily, hourly).
2. **Trading Volumes**: The amount of Bitcoin traded during the respective time intervals.
3. **Technical Indicators**: Calculate indicators like RSI, MACD, Moving Averages, etc., commonly used for trading decisions.

You can use APIs from **Binance**, **Coinbase Pro**, or **Yahoo Finance** to download this data.

### Step 0.2: Data Preparation

1. **Data Cleaning**:
   - Remove missing or anomalous values.
   - Convert time series data if necessary to ensure consistent time intervals.

2. **Calculate Technical Indicators**:
   - Add columns with calculated values for RSI, MACD, Moving Averages, and other relevant indicators.

3. **Define Target Variable**:
   - Define the target, such as whether the price will increase or decrease. You can create a binary variable (1 for "buy," 0 for "sell") based on price changes over a certain period (e.g., if the price increases by more than 1% in the next period).

### Step 0.3: Building the Predictive Model

1. **Model Selection**:
   - Use supervised learning models like **Random Forest**, **XGBoost**, or **SVM** to classify moments as "buy" or "sell."

2. **Training the Model**:
   - Split the data into training and testing sets.
   - Train the model on the training data and tune hyperparameters for optimal performance.

3. **Model Evaluation**:
   - Use metrics like **accuracy**, **F1 score**, **Precision**, and **Recall** to assess how well the model predicts "buy" and "sell" signals.

### Step 0.4: Strategy Simulation (Backtesting)

Before deploying the bot in a live environment, backtest it to assess its performance on historical data.

1. **Use the Test Dataset**: Evaluate how the bot would perform if buy and sell decisions had been made based on historical data.
2. **Evaluate the Strategy**: Calculate key metrics like:
   - **Return**: Compare achieved profit relative to a baseline (e.g., buy-and-hold).
   - **Maximum Drawdown**: Assess the largest losses during consecutive failed trades.
   - **Risk/Reward Ratio**.

### Step 0.5: Deployment in a Live Environment

1. **Choose a Trading Platform**:
   - Connect the bot to a trading API from exchanges like **Binance** or **Coinbase Pro**, which support live order execution.

2. **Build the Bot’s Core Logic**:
   - At each predefined interval (e.g., hourly), the bot should pull current data, calculate new indicators, and predict whether to buy or sell.
   - If the prediction is "buy," the bot sends a buy order; if "sell," a sell order.

3. **Risk Management Parameters**:
   - Define stop-loss and take-profit limits.
   - Set a maximum amount of funds the bot can use per trade, ensuring it does not take excessive risks.

### Step 0.6: Monitoring and Optimization

1. **Monitor Real-Time Bot Performance**:
   - Collect statistics on real trades to assess the model’s success in live conditions.

2. **Model Adjustment**:
   - Regularly update the data and retrain the model to accommodate new market conditions.
   - Test new indicators or models if optimization is required.

This strategy can be expanded by including other factors, such as sentiment analysis or blockchain transaction data.

## 1. Step 1: Data Collection

First, gather historical Bitcoin price data, which will be used for model training. Ideally, this data should include:

1. **Price Data**: Open, close, high, and low prices over specific time intervals (e.g., daily, hourly).
2. **Trading Volumes**: The amount of Bitcoin traded during the respective time intervals.
3. **Technical Indicators**: Calculate indicators like RSI, MACD, Moving Averages, etc., commonly used for trading decisions.

You can use APIs from **Binance**, **Coinbase Pro**, or **Yahoo Finance** to download this data.

### 1.1. Gather historical Bitcoin price data

**Price Data**: Gathered the open, close, high, and low prices over specific time intervals (every minute)

**Trading Volumes**: The amount of Bitcoin traded during the respective time intervals.

**TODO**: what ist Unit of the volume ????

In [2]:
btc_price_data_1_year = pd.read_csv("data/bitcoin_historical_data_1_year.csv")
btc_price_data_1_year

,timestamp,open,high,low,close,volume,date,time
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00
...,...,...,...,...,...,...,...,...
528627,2024-10-31 23:56:00,70238.76,70248.97,70248.97,70238.76,1.189134,2024-10-31,23:56:00
528628,2024-10-31 23:57:00,70218.00,70250.00,70238.77,70233.24,4.767082,2024-10-31,23:57:00
528629,2024-10-31 23:58:00,70193.97,70242.25,70232.55,70207.78,9.589688,2024-10-31,23:58:00
528630,2024-10-31 23:59:00,70175.16,70207.79,70207.79,70197.83,7.112237,2024-10-31,23:59:00


In [3]:
btc_price_data_1_year.describe().T

,count,mean,std,min,25%,50%,75%,max
open,528632.0,57052.764723,11061.821828,34079.460000,44083.780000,61015.800000,66127.655000,73718.320000
high,528632.0,57100.379874,11072.504219,34133.680000,44119.545000,61066.475000,66178.047500,73835.570000
low,528632.0,57076.624894,11067.091134,34113.930000,44103.265000,61041.025000,66153.502500,73815.030000
close,528632.0,57076.858348,11067.114019,34114.860000,44103.380000,61041.340000,66154.337500,73815.430000
volume,528632.0,8.915933,17.137792,0.001083,1.730652,4.020989,9.541108,1163.832604


In [4]:
btc_price_data_1_year.dtypes

timestamp     object
open         float64
high         float64
low          float64
close        float64
volume       float64
date          object
time          object
dtype: object

In [5]:
btc_price_data_1_year.isnull().sum()

timestamp    0
open         0
high         0
low          0
close        0
volume       0
date         0
time         0
dtype: int64

## 2. Step 2: Data Preparation

1. **Data Cleaning**:
   - Remove missing or anomalous values.
   - Convert time series data if necessary to ensure consistent time intervals.

2. **Calculate Technical Indicators**:
   - Add columns with calculated values for RSI, MACD, Moving Averages, and other relevant indicators.

3. **Define Target Variable**:
   - Define the target, such as whether the price will increase or decrease. You can create a binary variable (1 for "buy," 0 for "sell") based on price changes over a certain period (e.g., if the price increases by more than 1% in the next period).

### 2.1. Data Tidying and Cleaning

Convert the `timestamp` and`date`columns from *object* type to *datetime64* type.

**TODO** - Convert `time` - Pandas lacks a native time type, using datetime64[ns] with placeholder dates is often a suitable workaround for time operations.

In [6]:
btc_price_data_1_year.timestamp = pd.to_datetime(btc_price_data_1_year.timestamp)
btc_price_data_1_year.date = pd.to_datetime(btc_price_data_1_year.date)
# btc_price_data_1_year.time = pd.to_datetime(btc_price_data_1_year.time, format='%H:%M:%S').dt.time

btc_price_data_1_year.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
date         datetime64[ns]
time                 object
dtype: object

In [7]:
btc_price_data_1_year

,timestamp,open,high,low,close,volume,date,time
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00
...,...,...,...,...,...,...,...,...
528627,2024-10-31 23:56:00,70238.76,70248.97,70248.97,70238.76,1.189134,2024-10-31,23:56:00
528628,2024-10-31 23:57:00,70218.00,70250.00,70238.77,70233.24,4.767082,2024-10-31,23:57:00
528629,2024-10-31 23:58:00,70193.97,70242.25,70232.55,70207.78,9.589688,2024-10-31,23:58:00
528630,2024-10-31 23:59:00,70175.16,70207.79,70207.79,70197.83,7.112237,2024-10-31,23:59:00


In [8]:
btc_price_data_1_year.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
date         datetime64[ns]
time                 object
dtype: object

### 2.2. Calculate Target Variable

In [9]:
def create_target_variable(data, threshold = 0.01):
    """
    Computes and sets the 'target' variable from the input 'data' and 'threshold'.
    Creates a 'target' column with the computed values in the 'data' DataFrame.

    Parameters
    ----------
    data: a DataFrame with the time series data. There must be a column named 'close'! 
          This column will be used by the user to calculate the 'target' variable.
    
    threshold: threshold for the price change to classify as 'buy' or 'sell'. For instance, if you want a 1% increase to be a 'buy' signal,
               the threshold will be 0.01. Adjust this threshold as per your strategy.
    """
    # Create a copy of the DataFrame
    data_copy = data.copy(deep=True)
    
    # Compute the percentage change between the current close price and the close price in the next period.
    # This will help define whether there’s a significant increase or decrease.
    data_copy['future_return'] = ((data_copy['close'].shift(-1) - data_copy['close']) / data_copy['close']) * 100

    # Define the target as 1 (buy) if the future return is above the threshold, and 0 (sell) if it is below or equal to the threshold.
    data_copy['target'] = (data_copy['future_return'] > threshold).astype(float)

    # The last row in your dataset will have a NaN value for 'future_return' due to the shift operation. Drop this row to clean up the dataset.
    data_copy = data_copy.dropna()

    # Check the balance of 1s and 0s in our target variable to understand how many “buy” and “sell” signals we have.
    print(data_copy['target'].value_counts())

    return data_copy

In [10]:
# Define the threshold for the price change to classify as 'buy' or 'sell'. For instance, if we want a 1% increase to be a 'buy' signal,
# the threshold will be 0.01.
threshold = 0.01

# Compute the 'target' variable
btc_price_data_1_year = create_target_variable(btc_price_data_1_year, threshold)

target
0.0    317152
1.0    211479
Name: count, dtype: int64


In [11]:
btc_price_data_1_year.head(10)

,timestamp,open,high,low,close,volume,date,time,future_return,target
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00,-0.072286,0.0
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00,0.039662,1.0
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00,-0.078542,0.0
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00,-0.020416,0.0
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00,-0.045404,0.0
5,2023-11-01 00:05:00,34601.12,34629.21,34620.51,34606.55,4.765130,2023-11-01,00:05:00,0.046725,1.0
6,2023-11-01 00:06:00,34601.14,34630.82,34606.55,34622.72,3.357925,2023-11-01,00:06:00,0.124398,1.0
7,2023-11-01 00:07:00,34624.88,34678.59,34627.35,34665.79,13.859914,2023-11-01,00:07:00,-0.061992,0.0
8,2023-11-01 00:08:00,34637.70,34670.23,34660.89,34644.30,7.768562,2023-11-01,00:08:00,-0.071671,0.0
9,2023-11-01 00:09:00,34612.67,34665.75,34644.29,34619.47,4.488087,2023-11-01,00:09:00,-0.046997,0.0


### 2.3. Calculate Technical Indicators:
   - Add columns with calculated values for RSI, MACD, Moving Averages, and other relevant indicators.

#### **TODO** - **Technical Indicators**: Calculate indicators like RSI, MACD, Moving Averages, etc., commonly used for trading decisions.

In [12]:
btc_price_data_1_year.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'date', 'time',
       'future_return', 'target'],
      dtype='object')

**TODO** - Use python libraties for the calculations

#### Moving Average Convergence Divergence (MACD)

In [13]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    data['EMA12'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['close'].ewm(span=long_window, adjust=False).mean()
    
    # MACD Line
    data['MACD'] = data['EMA12'] - btc_price_data_1_year['EMA26']
    
    # Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    return data

In [14]:
# Assuming 'data' is a DataFrame with a 'Close' price column
btc_price_data_1_year = calculate_macd(btc_price_data_1_year)
btc_price_data_1_year[['close', 'MACD', 'Signal_Line']]

,close,MACD,Signal_Line
0,34667.88,0.000000,0.000000
1,34642.82,-1.999088,-0.399818
2,34656.56,-2.446476,-0.809149
3,34629.34,-4.940509,-1.635421
4,34622.27,-7.402210,-2.788779
...,...,...,...
528626,70248.97,-47.477702,-35.505422
528627,70238.76,-50.163495,-38.437037
528628,70233.24,-52.136426,-41.176915
528629,70207.78,-55.119020,-43.965336


In [15]:
btc_price_data_1_year.head(10)

,timestamp,open,high,low,close,volume,date,time,future_return,target,EMA12,EMA26,MACD,Signal_Line
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00,-0.072286,0.0,34667.880000,34667.880000,0.000000,0.000000
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00,0.039662,1.0,34664.024615,34666.023704,-1.999088,-0.399818
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00,-0.078542,0.0,34662.876213,34665.322689,-2.446476,-0.809149
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00,-0.020416,0.0,34657.716796,34662.657304,-4.940509,-1.635421
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00,-0.045404,0.0,34652.263442,34659.665652,-7.402210,-2.788779
5,2023-11-01 00:05:00,34601.12,34629.21,34620.51,34606.55,4.765130,2023-11-01,00:05:00,0.046725,1.0,34645.230605,34655.731159,-10.500554,-4.331134
6,2023-11-01 00:06:00,34601.14,34630.82,34606.55,34622.72,3.357925,2023-11-01,00:06:00,0.124398,1.0,34641.767435,34653.285888,-11.518453,-5.768598
7,2023-11-01 00:07:00,34624.88,34678.59,34627.35,34665.79,13.859914,2023-11-01,00:07:00,-0.061992,0.0,34645.463214,34654.212119,-8.748904,-6.364659
8,2023-11-01 00:08:00,34637.70,34670.23,34660.89,34644.30,7.768562,2023-11-01,00:08:00,-0.071671,0.0,34645.284258,34653.477888,-8.193629,-6.730453
9,2023-11-01 00:09:00,34612.67,34665.75,34644.29,34619.47,4.488087,2023-11-01,00:09:00,-0.046997,0.0,34641.312834,34650.958785,-9.645951,-7.313553


#### Using pandas_ta to calculate RSI, SMA, EMA and MACD

In [16]:
# Install pandas_ta if not already installed
# !pip install pandas_ta

import pandas as pd
import pandas_ta as ta

# Assuming 'data' is a DataFrame with a 'Close' price column
btc_price_data_1_year['RSI'] = ta.rsi(btc_price_data_1_year['close'], length=14)
# btc_price_data_1_year['MACD'], btc_price_data_1_year['Signal_Line'], _ = ta.macd(btc_price_data_1_year['close'], fast=12, slow=26, signal=9)
btc_price_data_1_year['SMA'] = ta.sma(btc_price_data_1_year['close'], length=20)
btc_price_data_1_year['EMA'] = ta.ema(btc_price_data_1_year['close'], length=20)

# print(btc_price_data_1_year[['close', 'RSI','SMA', 'EMA']])
print(btc_price_data_1_year[['close', 'RSI', 'MACD', 'Signal_Line', 'SMA', 'EMA']])


           close        RSI       MACD  Signal_Line         SMA           EMA
0       34667.88        NaN   0.000000     0.000000         NaN           NaN
1       34642.82        NaN  -1.999088    -0.399818         NaN           NaN
2       34656.56        NaN  -2.446476    -0.809149         NaN           NaN
3       34629.34        NaN  -4.940509    -1.635421         NaN           NaN
4       34622.27        NaN  -7.402210    -2.788779         NaN           NaN
...          ...        ...        ...          ...         ...           ...
528626  70248.97  27.271955 -47.477702   -35.505422  70367.7305  70348.638181
528627  70238.76  26.231313 -50.163495   -38.437037  70352.5490  70338.173592
528628  70233.24  25.661202 -52.136426   -41.176915  70338.1685  70328.179916
528629  70207.78  23.160865 -55.119020   -43.965336  70325.1425  70316.713258
528630  70197.83  22.248494 -57.621405   -46.696549  70313.8910  70305.391043

[528631 rows x 6 columns]


In [17]:
btc_price_data_1_year

,timestamp,open,high,low,close,volume,date,time,future_return,target,EMA12,EMA26,MACD,Signal_Line,RSI,SMA,EMA
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00,-0.072286,0.0,34667.880000,34667.880000,0.000000,0.000000,NaN,NaN,NaN
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00,0.039662,1.0,34664.024615,34666.023704,-1.999088,-0.399818,NaN,NaN,NaN
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00,-0.078542,0.0,34662.876213,34665.322689,-2.446476,-0.809149,NaN,NaN,NaN
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00,-0.020416,0.0,34657.716796,34662.657304,-4.940509,-1.635421,NaN,NaN,NaN
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00,-0.045404,0.0,34652.263442,34659.665652,-7.402210,-2.788779,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528626,2024-10-31 23:55:00,70248.97,70264.97,70248.98,70248.97,1.604753,2024-10-31,23:55:00,-0.014534,0.0,70316.514762,70363.992465,-47.477702,-35.505422,27.271955,70367.7305,70348.638181
528627,2024-10-31 23:56:00,70238.76,70248.97,70248.97,70238.76,1.189134,2024-10-31,23:56:00,-0.007859,0.0,70304.552491,70354.715986,-50.163495,-38.437037,26.231313,70352.5490,70338.173592
528628,2024-10-31 23:57:00,70218.00,70250.00,70238.77,70233.24,4.767082,2024-10-31,23:57:00,-0.036251,0.0,70293.581339,70345.717765,-52.136426,-41.176915,25.661202,70338.1685,70328.179916
528629,2024-10-31 23:58:00,70193.97,70242.25,70232.55,70207.78,9.589688,2024-10-31,23:58:00,-0.014172,0.0,70280.381133,70335.500152,-55.119020,-43.965336,23.160865,70325.1425,70316.713258


In [18]:
btc_price_data_1_year.shape

(528631, 17)

In [19]:
btc_price_data_1_year.dtypes

timestamp        datetime64[ns]
open                    float64
high                    float64
low                     float64
close                   float64
volume                  float64
date             datetime64[ns]
time                     object
future_return           float64
target                  float64
EMA12                   float64
EMA26                   float64
MACD                    float64
Signal_Line             float64
RSI                     float64
SMA                     float64
EMA                     float64
dtype: object

#### TODO - What should we do with the time (object type)? Do we need this column ???? 

In [20]:
btc_price_data_1_year.isnull().sum()

timestamp         0
open              0
high              0
low               0
close             0
volume            0
date              0
time              0
future_return     0
target            0
EMA12             0
EMA26             0
MACD              0
Signal_Line       0
RSI              14
SMA              19
EMA              19
dtype: int64

#### TODO - Remove all NaN ????

## 3. Step 3: Building the Predictive Model

1. **Model Selection**:
   - Use supervised learning models like **Random Forest**, **XGBoost**, or **SVM** to classify moments as "buy" or "sell."

2. **Training the Model**:
   - Split the data into training and testing sets.
   - Train the model on the training data and tune hyperparameters for optimal performance.

3. **Model Evaluation**:
   - Use metrics like **accuracy**, **F1 score**, **Precision**, and **Recall** to assess how well the model predicts "buy" and "sell" signals.



Here is a detailed step-by-step procedure for creating a model to predict Bitcoin trading signals using machine learning:

### 3.1. Step 1: Data Preparation

1. **Data Splitting**:
   - Use a portion of historical data as a training set and the remaining portion as a test set (e.g., 80/20 split).

2. **Creating the Target Variable**:
   - Create a target variable for the model, such as "buy" or "sell."
   - Example: Define "buy" (1) if the price is expected to rise by more than 1% within the next 24 hours, and "sell" (0) otherwise.

3. **Adding Technical Indicators**:
   - Calculate indicators like **RSI**, **MACD**, **Bollinger Bands**, **Moving Averages**, and add them as columns to the dataset. These will be used as features for training the model.

4. **Scaling the Data**:
   - For algorithms like SVM and KNN, scale feature values to a range of 0 to 1 or -1 to 1. Use `StandardScaler` or `MinMaxScaler` from `scikit-learn`.

### 3.2. Step 2: Model Selection

1. **Choosing an Algorithm**:
   - Select a model suitable for **binary classification**, such as:
     - **Random Forest**: Capable of capturing complex, nonlinear patterns in data.
     - **XGBoost**: A boosting algorithm that often achieves high accuracy by combining weak classifiers.
     - **SVM (Support Vector Machine)**: Useful when data is well-scaled and a model with good generalization ability is needed.

2. **Implementing the Model**:
   - Import the chosen model from `scikit-learn` (e.g., `RandomForestClassifier`, `XGBClassifier`, or `SVC`).
   - Set initial hyperparameters (start with default values and optimize later).

Example code:

```python
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
```

### 3.3. Step 3: Training the Model

1. **Training the Model**:
   - Train the model using the training dataset. Split the data into X (features) and y (target variable).
   
   ```python
   model.fit(X_train, y_train)
   ```

2. **Hyperparameter Optimization (Optional)**:
   - Use `GridSearchCV` or `RandomizedSearchCV` to tune hyperparameters. This can help the model achieve better results.

### 3.4. Step 4: Model Evaluation

1. **Making Predictions**:
   - Use the model to make predictions on the test dataset.

   ```python
   y_pred = model.predict(X_test)
   ```

2. **Evaluating Performance**:
   - Use metrics for binary model evaluation, such as accuracy, precision, recall, and F1-score.
   
   ```python
   from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
   
   accuracy = accuracy_score(y_test, y_pred)
   precision = precision_score(y_test, y_pred)
   recall = recall_score(y_test, y_pred)
   f1 = f1_score(y_test, y_pred)
   
   print(f'Accuracy: {accuracy:.2f}')
   print(f'Precision: {precision:.2f}')
   print(f'Recall: {recall:.2f}')
   print(f'F1 Score: {f1:.2f}')
   ```

### 3.5. Step 5: Saving the Model (Optional)

Once the model is trained and has achieved the desired accuracy, you can save it for later use with `joblib` or `pickle`.

```python
import joblib
joblib.dump(model, 'trading_model.pkl')
```

### 3.6. Step 6: Final Testing

After training, conduct additional tests with new (unseen) data to ensure the model generalizes well to new scenarios.

Afterwards, you can use this model for real-time predictions in a trading bot.

## 4. Step 4: Strategy Simulation (Backtesting)

Before deploying the bot in a live environment, backtest it to assess its performance on historical data.

1. **Use the Test Dataset**: Evaluate how the bot would perform if buy and sell decisions had been made based on historical data.
2. **Evaluate the Strategy**: Calculate key metrics like:
   - **Return**: Compare achieved profit relative to a baseline (e.g., buy-and-hold).
   - **Maximum Drawdown**: Assess the largest losses during consecutive failed trades.
   - **Risk/Reward Ratio**.